In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
import swifter
import nltk
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Embedding, Conv2D, MaxPooling2D, Dropout,Reshape, Flatten,LSTM
from tensorflow.keras import regularizers
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import classification_report


In [81]:
NUM_WORKERS = 8
TEST_SIZE = 0.3
LABEL_COLUMN_NAME = "oh_label"
TEXT_COLUMN_NAME = "Text"
DATASET_PATH = "../Data/ver1.csv"

In [93]:
df= pd.read_csv(DATASET_PATH, index_col=False)
df = df.dropna()
df.drop(["Unnamed: 0", "index"],axis=1, inplace = True)

In [89]:
from gensim.models.keyedvectors import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
import re
def word2vecMateix(tokens_list):
    vectorized = np.array([word2vec.get_vector(word) for word in tokens_list if word in word2vec ])
    if len(vectorized) < 100:
            vectorized = np.concatenate([vectorized,  np.zeros((100 - len(vectorized), 300))], axis = None)
    if len(vectorized) > 100:
            vectorized = vectorized[:100]
    return pd.Series(vectorized.flatten())
regex = re.compile('[^a-z ]')
df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].swifter.apply(lambda text: word2vecMateix(regex.sub('',text.lower()).split()))

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

In [96]:
df[TEXT_COLUMN_NAME]

0        Im 12 and i can understand it perfectly. You s...
1        #mkr Boy, @FourinHandyou sure know how to dish...
2          Fuck you all. This site is full of stuck up ...
3        `   ::I don't disagree with your point, except...
4          do i look like i give a fuck, u are all sad ...
                               ...                        
99995    `  == d15b8 - just you leave it alone ==  ``co...
99996     :Thanks for the feedback, and I'm very glad t...
99997      ==Fast and Furious 7== Please do your resear...
99998                            @KWiebs31 ABLOO BLOO BLOO
99999    `  Clearly you know very little about these ba...
Name: Text, Length: 100000, dtype: object

In [ ]:
# import re
# df= pd.read_csv(DATASET_PATH, index_col=False)
# df = df.dropna()
# df.drop(["Unnamed: 0", "index"],axis=1, inplace = True)
# regex = re.compile('[^a-z ]')
# df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].swifter.apply(lambda text: regex.sub('',text.lower()))
# lens = df[TEXT_COLUMN_NAME].swifter.apply(len)
# df = df[lens < lens.quantile(.75)]
# print(df[LABEL_COLUMN_NAME].value_counts()/df.shape[0])
# mask = df[TEXT_COLUMN_NAME].swifter.apply(lambda text: all([word in word2vec for word in text.split()]))
# df[mask][LABEL_COLUMN_NAME].value_counts()

In [42]:
X,y = df[TEXT_COLUMN_NAME],df[LABEL_COLUMN_NAME]
stopwords = nltk.corpus.stopwords.words('english')
X = X.swifter.apply(lambda text: ' '.join([item for item in nltk.word_tokenize(text) if item not in stopwords]))
X = X.str.replace('[^\w\s]','')

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

In [51]:
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

In [53]:
not_found = set()
for word in word_index:
    if word not in word_vectors:
        not_found.add(word)

In [54]:
X = X.swifter.apply(lambda text: ' '.join([item for item in text.split() if item.lower() not in not_found]))

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

In [38]:
# from collections import Counter
# results = Counter()
# X.str.lower().str.split().apply(results.update)
# counts = pd.DataFrame.from_dict(results, orient='index').reset_index()
# counts.columns = ["index","count"]
# counts = counts.sort_values("count",ascending=False).reset_index(drop=True)
# counts['count'].value_counts()

In [79]:
X_train,X_test, y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=93)

In [59]:
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index

In [60]:
not_found = set()
for word in word_index.keys():
    try:
        embedding_vector = word_vectors[word]
    except KeyError:
        not_found.add(word)


In [61]:
len(not_found)

2332

In [62]:
EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1 - len(not_found)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
i = 0
for word in word_index.keys():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]
        i+=1


In [63]:
X = X.swifter.apply(lambda text: ' '.join([item for item in text.split() if item not in not_found]))

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

In [78]:
embedding_matrix.shape

(50663, 300)

In [67]:
X_train = pad_sequences(X_train,padding='post')
X_test = pad_sequences(X_test,maxlen=X_train.shape[1],padding='post')

In [70]:
X_train.shape

(80000, 2809)

In [73]:
sequence_length = X_train.shape[1]
model = Sequential()
model.add(Embedding(vocabulary_size,EMBEDDING_DIM,weights=[embedding_matrix],trainable=False,input_length=sequence_length))
model.add(Reshape((sequence_length,EMBEDDING_DIM,1)))
model.add(Conv2D(100, (3, EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l1_l2(0.01)))
model.add(MaxPooling2D((sequence_length - 4, 1), strides=(1,1)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(150, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=["accuracy",tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2809, 300)         15198900  
_________________________________________________________________
reshape_7 (Reshape)          (None, 2809, 300, 1)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 2807, 1, 100)      90100     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 1, 100)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 300)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 150)              

In [74]:
weights = 1 - pd.Series(y).value_counts()/y.size
weights = weights.to_dict()
weights

{0.0: 0.26771, 1.0: 0.73229}

In [76]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
time_stopping_callback = tfa.callbacks.TimeStopping(seconds=60*60*15, verbose=1) 
history = model.fit(X_train,y_train,batch_size=200,epochs=1000,
                    validation_split=0.1,class_weight=weights,
                    use_multiprocessing=True,workers=8,callbacks=[callback,time_stopping_callback],shuffle=True)

Epoch 1/1000
  1/360 [..............................] - ETA: 14:07 - loss: 3.9440 - accuracy: 0.6250 - recall_6: 0.2000 - precision_6: 0.1875

InvalidArgumentError:  indices[93,2805] = 51358 is not in [0, 50663)
	 [[node sequential_7/embedding_7/embedding_lookup (defined at <ipython-input-76-c30c230dd42d>:3) ]] [Op:__inference_train_function_10533]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_7/embedding_7/embedding_lookup:
 sequential_7/embedding_7/embedding_lookup/10009 (defined at C:\ProgramData\Anaconda3\lib\contextlib.py:113)

Function call stack:
train_function


In [ ]:
model.save("cnn_word2vec_second_chance")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Train Metrics')
plt.ylabel('Metric')
plt.xlabel('Epoch')
plt.legend(['accuracy', 'val_accuracy','loss','val_loss'])
plt.show()
y_pred_test = model.predict(X_test)
y_pred_test[y_pred_test > 0.5] = 1
y_pred_test[y_pred_test <= 0.5] = 0
print("Test: \n")
print(classification_report(y_test.argmax(axis=1),y_pred_test))
print("\nTrain: \n")
y_pred_train = model.predict(X_train)
y_pred_train[y_pred_train > 0.5] = 1
y_pred_train[y_pred_train <= 0.5] = 0
print(classification_report(y_train.argmax(axis=1),y_pred_train))